In [24]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import itertools
import pickle

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")

cols_to_drop = ['VERBALE', 'DATA']
X = df.drop(columns=cols_to_drop)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
X.head

<bound method NDFrame.head of      SESSO  ANNI  PESO  ALTEZZA        BMI  Mezzo  Testa:Neurocranio  \
0      0.0  81.0  84.0    175.0  27.428571    0.0                1.0   
1      1.0  69.0  69.0    162.0  26.291724    1.0                4.0   
2      1.0  71.0  67.0    155.0  27.887617    1.0                2.0   
3      1.0  54.0  60.0    159.0  23.733238    1.0                4.0   
4      1.0  78.0  69.0    167.0  24.740937    1.0                2.0   
..     ...   ...   ...      ...        ...    ...                ...   
125    1.0  82.0  80.0    162.0  30.559366    0.0                1.0   
126    1.0  70.0  75.0    157.0  30.670615    0.0                2.0   
127    0.0  45.0  45.0    177.0  14.363689    1.0                0.0   
128    0.0  54.0  89.0    168.0  31.746032    0.0                0.0   
129    1.0  86.0  50.0    155.0  20.811655    0.0                1.0   

     Testa:Splancnocranio  Testa:Telencefalo  Testa:Cervelletto  ...  \
0                     0.0        

In [3]:
def nested_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [4]:
results = nested_cv_accuracy(X)

In [5]:
results

{'algorythm': 'OneClassSVM',
 'best kernel': 'linear',
 'best gamma': 1e-09,
 'best nu': 0.99,
 'method used for model selection': 'accuracy on nested cv',
 'accuracy mean': 50.61538461538462,
 'accuracy std': 6.122999612963814,
 'precision mean': 45.454739601643624,
 'precision  std': 26.340900801262872,
 'recall mean': 33.285714285714285,
 'recall std': 29.33306276931496,
 'data set': 'no const, no equals'}

In [6]:
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123,45.45474,26.340901,33.285714,29.333063,"no const, no equals"


In [7]:
def nested_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 score on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [8]:
results = nested_cv_f1(X)

In [9]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [10]:
scores_df = add_record(scores_df, results)

In [11]:
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"


In [12]:
def outer_cv_inner_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [13]:
results = outer_cv_inner_holdout_accuracy(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"
2,OneClassSVM,rbf,1.000000e-09,0.05,accuracy on outer cv inner holdout,45.000000,6.583555,18.766668,26.065744,16.428571,26.620922,"no const, no equals"


In [14]:
def outer_cv_inner_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [15]:
results = outer_cv_inner_holdout_f1(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"
2,OneClassSVM,rbf,1.000000e-09,0.05,accuracy on outer cv inner holdout,45.000000,6.583555,18.766668,26.065744,16.428571,26.620922,"no const, no equals"
3,OneClassSVM,rbf,1.000000e-04,0.15,f1 on outer cv inner holdout,55.692308,11.100402,59.901920,20.849513,50.142857,28.553208,"no const, no equals"


In [16]:
def outer_holdout_inner_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0

        for train_idx, test_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[train_idx], X_train[test_idx]
            y_trainval, y_valid = y_train[train_idx], y_train[test_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [17]:
results = outer_holdout_inner_cv_accuracy(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"
2,OneClassSVM,rbf,1.000000e-09,0.05,accuracy on outer cv inner holdout,45.000000,6.583555,18.766668,26.065744,16.428571,26.620922,"no const, no equals"
3,OneClassSVM,rbf,1.000000e-04,0.15,f1 on outer cv inner holdout,55.692308,11.100402,59.901920,20.849513,50.142857,28.553208,"no const, no equals"
4,OneClassSVM,rbf,1.000000e-09,0.04,accuracy on outer holdout inner cv,46.538462,9.174508,25.375458,27.856669,11.771709,19.117465,"no const, no equals"


In [18]:
def outer_holdout_inner_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0

        for train_idx, test_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[train_idx], X_train[test_idx]
            y_trainval, y_valid = y_train[train_idx], y_train[test_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [19]:
results = outer_holdout_inner_cv_f1(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"
2,OneClassSVM,rbf,1.000000e-09,0.05,accuracy on outer cv inner holdout,45.000000,6.583555,18.766668,26.065744,16.428571,26.620922,"no const, no equals"
3,OneClassSVM,rbf,1.000000e-04,0.15,f1 on outer cv inner holdout,55.692308,11.100402,59.901920,20.849513,50.142857,28.553208,"no const, no equals"
4,OneClassSVM,rbf,1.000000e-09,0.04,accuracy on outer holdout inner cv,46.538462,9.174508,25.375458,27.856669,11.771709,19.117465,"no const, no equals"
5,OneClassSVM,rbf,1.000000e-09,0.08,f1 on outer holdout inner cv,58.461538,8.391317,66.417915,18.385012,44.706710,22.064563,"no const, no equals"


In [20]:
def double_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [21]:
results = double_holdout_accuracy(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"
2,OneClassSVM,rbf,1.000000e-09,0.05,accuracy on outer cv inner holdout,45.000000,6.583555,18.766668,26.065744,16.428571,26.620922,"no const, no equals"
3,OneClassSVM,rbf,1.000000e-04,0.15,f1 on outer cv inner holdout,55.692308,11.100402,59.901920,20.849513,50.142857,28.553208,"no const, no equals"
4,OneClassSVM,rbf,1.000000e-09,0.04,accuracy on outer holdout inner cv,46.538462,9.174508,25.375458,27.856669,11.771709,19.117465,"no const, no equals"
5,OneClassSVM,rbf,1.000000e-09,0.08,f1 on outer holdout inner cv,58.461538,8.391317,66.417915,18.385012,44.706710,22.064563,"no const, no equals"
6,OneClassSVM,rbf,1.000000e-09,0.01,accuracy on double holdout,44.615385,6.923077,16.333333,30.566867,7.708333,12.220841,"no const, no equals"


In [22]:
def double_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [23]:
results = double_holdout_f1(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.99,accuracy on nested cv,50.615385,6.123000,45.454740,26.340901,33.285714,29.333063,"no const, no equals"
1,OneClassSVM,rbf,1.000000e-04,0.47,f1 score on nested cv,57.538462,7.918221,61.840678,13.544671,53.857143,19.959653,"no const, no equals"
2,OneClassSVM,rbf,1.000000e-09,0.05,accuracy on outer cv inner holdout,45.000000,6.583555,18.766668,26.065744,16.428571,26.620922,"no const, no equals"
3,OneClassSVM,rbf,1.000000e-04,0.15,f1 on outer cv inner holdout,55.692308,11.100402,59.901920,20.849513,50.142857,28.553208,"no const, no equals"
4,OneClassSVM,rbf,1.000000e-09,0.04,accuracy on outer holdout inner cv,46.538462,9.174508,25.375458,27.856669,11.771709,19.117465,"no const, no equals"
5,OneClassSVM,rbf,1.000000e-09,0.08,f1 on outer holdout inner cv,58.461538,8.391317,66.417915,18.385012,44.706710,22.064563,"no const, no equals"
6,OneClassSVM,rbf,1.000000e-09,0.01,accuracy on double holdout,44.615385,6.923077,16.333333,30.566867,7.708333,12.220841,"no const, no equals"
7,OneClassSVM,rbf,1.000000e-03,0.31,f1 on double holdout,58.846154,9.888431,60.690698,22.427831,57.725975,25.924242,"no const, no equals"


In [25]:
file_path = 'svm_exp1.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)